<a href="https://colab.research.google.com/github/stygian2a/RoombaGym/blob/master/Roomba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO


In [10]:
!pip uninstall keras -y
!pip install keras==2.2.4 

!pip uninstall tensorflow -y
!pip install tensorflow==1.13.1 

#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Uninstalling Keras-2.2.4:
  Successfully uninstalled Keras-2.2.4
  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl


Uninstalling tensorflow-1.13.1:
  Successfully uninstalled tensorflow-1.13.1
  Using cached https://files.pythonhosted.org/packages/77/63/a9fa76de8dffe7455304c4ed635be4aa9c0bacef6e0633d87d5f54530c5c/tensorflow-1.13.1-cp36-cp36m-manylinux1_x86_64.whl


In [11]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!apt-get install x11-utils > /dev/null 2>&1
!pip install keras-rl

!wget https://raw.githubusercontent.com/stygian2a/RoombaGym/master/rendering.py

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (45.2.0)
--2020-02-22 16:32:41--  https://raw.githubusercontent.com/stygian2a/RoombaGym/master/rendering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11819 (12K) [text/plain]
Saving to: ‘rendering.py.4’

rendering.py.4      100%[===================>]  11.54K  --.-KB/s    in 0s      

2020-02-22 16:32:41 (192 MB/s) - ‘rendering.py.4’ saved [11819/11819]



In [0]:
import gym
from gym import spaces
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython.display import clear_output
from time import sleep
from scipy.stats import multivariate_normal

from IPython import display as ipythondisplay

In [13]:
!dpkg --configure -a
!apt-get install xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 102 not upgraded.


In [14]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1005'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
class Roomba():
  def __init__(self, pos, direction, radius, battery, turn_angle=np.pi/9):
    self.pos = np.array(pos)
    self.direction = direction
    self.radius = radius
    self.turn_angle = turn_angle
    self.battery = battery
    self.max_battery = battery

  def set_pos(self, pos):
    self.pos = np.array(pos)

  def set_direction(self, direction):
    self.direction = np.array(direction)

  def get_forward(self, direction=None):
    if direction == None:
      direction = self.direction
    new_x = np.sin(direction) + self.pos[0]
    new_y = np.cos(direction) + self.pos[1]
    return [new_x, new_y]

  def get_backward(self, direction=None):
    if direction == None:
      direction = self.direction
    new_x = np.sin(direction+np.pi) + self.pos[0]
    new_y = np.cos(direction+np.pi) + self.pos[1]
    return [new_x, new_y]

  def turn_right(self):
    return (self.direction - self.turn_angle) % (2*np.pi)

  def turn_left(self):
    return (self.direction + self.turn_angle) % (2*np.pi)

  def decrease_battery(self):
    self.battery -=1

  def recharge_battery(self):
    self.battery = self.max_battery

  def get_life(self):
    return self.battery / self.max_battery


  #https://stackoverflow.com/questions/49551440/python-all-points-on-circle-given-radius-and-center
  def get_cases_underneath(self, xmax, ymax, pos = None):
    if pos == None:
      pos = self.pos
    radius = self.radius
    x0 = pos[0] - 0.5
    y0 = pos[1] - 0.5

    L = []
    x_ = np.arange(x0 - radius - 1, x0 + radius + 1, dtype=int)
    y_ = np.arange(y0 - radius - 1, y0 + radius + 1, dtype=int)
    x, y = np.where((x_[:,np.newaxis] - x0)**2 + (y_ - y0)**2 <= radius**2)
    for x, y in zip(x_[x], y_[y]):
      if x>=0 and y>=0 and x<xmax and y<ymax:
        L.append((x, y))
    return L

In [0]:
class RoombaEnv(gym.Env):

  metadata = {
      'render.modes': ['human', 'rgb_array'],
      'video.frames_per_second' : 50
  }

  def __init__(self, height, width, battery, roomba_radius=2):
    super(RoombaEnv, self).__init__()

    self.action_space = spaces.Discrete(4) #Forward, Backwards, Right, Left

    """                                      
    self.observation_space = spaces.Tuple([
        spaces.Box(low=0, high=4, shape=(height, width), dtype=np.float32), #1=cleanable, 2=wall, 3=charging station
        spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([height, width, 2*np.pi, 1]), dtype=np.float32)
    ])
    """
    #self.observation_space = spaces.Box(low=np.array([0, 0, 0, 0]), high=np.array([height, width, 2*np.pi, 1]), dtype=np.float32)
    self.observation_space = spaces.Box(low=np.array(np.zeros(4+height*width)), high=np.concatenate([np.array([height, width, 2*np.pi, 1]), np.ones(height*width)]), dtype=np.float32)

    self.width = width
    self.height = height
    self.battery = battery
    self.roomba_radius = roomba_radius

    self.cases_indexes = np.array( [(i,j) for i in range(height) for j in range(width)] )
    self.dirt_probabilities = multivariate_normal.pdf(self.cases_indexes, mean=[height/2,width/2], cov=10*np.diag([1,1]))
    self.dirt_probabilities = self.dirt_probabilities / np.sum(self.dirt_probabilities)

    self.room = np.ones((height, width))
    #obstacles = [[5,5], [5,6], [5,7], [5,8], [6,5], [7,5]]
    obstacles = []
    for obstIndex in obstacles:
      self.room[obstIndex[0],obstIndex[1]]= 2
    for i in range(4):
      for j in range(4):
        self.room[height-i-1, width-j-1] = 3
    self.viewer = None
    self.init_env()

  def init_env(self):
    self.roomba = Roomba( [self.height-3, self.width-3], -np.pi/2, self.roomba_radius, self.battery)
    self.done = False
    self.iteration = 0
    self.sum_rewards = 0
    self.dirty = np.zeros((self.height, self.width))


  def clean_floor(self):
    cases_underneath = self.roomba.get_cases_underneath(self.height, self.width)
    reward = 0
    for case_underneath in cases_underneath:
      if self.room[case_underneath] == 1:
        reward += min(1,self.dirty[case_underneath])
        self.dirty[case_underneath] = 0

      if self.room[case_underneath] == 3:
        self.roomba.recharge_battery()
    return reward

  def soil_floor(self):
    i = np.random.choice(self.height * self.width, p=self.dirt_probabilities)
    self.dirty[self.cases_indexes[i][0], self.cases_indexes[i][1]] = 1

  def step(self, action):
    self.iteration+=1
    #if random.random() <0.1:
    #  action = random.randint(0,3)
    #current_reward = 1
    current_reward = 0
    if action == 0:
      new_pos = self.roomba.get_forward()
      if self.is_safe(new_pos):
        self.roomba.set_pos(new_pos)
      #else:
      #  current_reward += -10

    elif action == 1:
      new_pos = self.roomba.get_backward()
      if self.is_safe(new_pos):
        self.roomba.set_pos(new_pos)
      #else:
      #  current_reward += -10

    elif action == 2:
      self.roomba.set_direction(self.roomba.turn_right())

    elif action == 3:
      self.roomba.set_direction(self.roomba.turn_left())
    self.roomba.decrease_battery()
      
    current_reward += self.clean_floor()
    
    #cost of living
    #current_reward -= 1

    self.soil_floor()

    if self.roomba.battery <= 0:
      current_reward += -200

    self.sum_rewards += current_reward

    if self.iteration>=10000 or self.roomba.battery <= 0:
      self.done = True
      
    return self.get_observation(),current_reward,self.done,{}

  def is_safe(self, new_pos):
    x = new_pos[0]
    y = new_pos[1]

    if y < 0 or x < 0 or x >= self.height or y >= self.width:
      return False

    cases_underneath = self.roomba.get_cases_underneath(self.height, self.width, new_pos)
    for case_underneath in cases_underneath:
      if self.room[case_underneath] == 2:
        return False

    return True

  def get_observation(self):
    #return np.array([self.roomba.pos[0]/self.height, self.roomba.pos[1]/self.width, self.roomba.direction, self.roomba.get_life()])
    return np.concatenate([np.array([self.roomba.pos[0]/self.height, self.roomba.pos[1]/self.width, self.roomba.direction, self.roomba.get_life()]), self.dirty.flatten()])

    #return (self.dirty, np.array([self.roomba.pos[0], self.roomba.pos[1], self.roomba.direction, self.roomba.get_life()]))

  def reset(self):
    self.init_env()
    return self.get_observation()

  def render(self, mode='human', close=False):
    scale = 6

    screen_width = scale*self.width
    screen_height = scale*self.height


    if self.viewer is None:
      #TODO: remove when it works
      import importlib
      import rendering
      importlib.reload(rendering)

      self.viewer = rendering.Viewer(screen_width, screen_height)

      self.cases = []
      for i in range(self.height):
        self.cases.append([])
        for j in range(self.width):
          x0 = i*scale
          y0 = j*scale
          x1 = (i+1)*scale
          y1 = (j+1)*scale
          case = rendering.FilledPolygon([(y0,x0), (y0,x1), (y1,x1), (y1,x0)])
          self.cases[i].append(case)
          self.viewer.add_geom(case)
      
      circle = rendering.make_circle(radius=self.roomba.radius*scale)
      circle.set_color(1,0,0)
      self.circle=circle
      direction_circle = rendering.make_circle(radius=self.roomba.radius*scale/2) #color?
      direction_circle.add_attr(rendering.Transform(translation=(self.roomba.radius*scale/2,0)))

      tracker = rendering.Compound([circle, direction_circle])

      self.tracker_trans = rendering.Transform()
      tracker.add_attr(self.tracker_trans)
      self.viewer.add_geom(tracker)

      
    for i in range(self.height):
        for j in range(self.width):
          if self.room[i,j] == 0:
            self.cases[i][j].set_color(1,1,1)
          elif self.room[i,j] == 1:
            self.cases[i][j].set_color(1 - 0.5 * min(1,self.dirty[i,j]),1 - 0.5 * min(1,self.dirty[i,j]),1 - 0.5 * min(1,self.dirty[i,j]))
          elif self.room[i,j] == 2:
            self.cases[i][j].set_color(0,0,0)
          elif self.room[i,j] == 3:
            self.cases[i][j].set_color(0.5,0.5,0)


    self.tracker_trans.set_translation(self.roomba.pos[1]*scale, self.roomba.pos[0]*scale)
    self.tracker_trans.set_rotation(self.roomba.direction)
    self.circle.set_color(self.roomba.get_life(),0,0)


    return self.viewer.render(return_rgb_array = mode=='rgb_array')

In [0]:
#TODO : Think of BETTER FEATURES (better value for turning)

Height = 20
Width = 20
battery = 500
pad = 2

env = wrap_env(RoombaEnv(Height, Width, battery, pad))
possible_actions = [env.roomba.get_forward, env.roomba.get_backward, env.roomba.turn_right, env.roomba.turn_left]

def closestDirt(x, y, mat):
  d = Height+Width
  for k in range(Height): 
    for l in range(Width):  
        if (mat[k][l] == 1): 
            d = min(d, abs(x - k) + abs(y - l))
  return d

def feature1(state, action):
  dirtMap = state[4:].reshape(20,20)
  pos = action()
  turn = isinstance(pos, float)
  if turn:
    pos=possible_actions[0](pos)
  x_min = max(0,int(pos[0])-pad)
  x_max = min(Height,int(pos[0])+pad)
  y_min = max(0,int(pos[1])-pad)
  y_max = min(Width,int(pos[1])+pad)
  AAA = np.sum(dirtMap[x_min:x_max, y_min:y_max])/(pad*pad)
  AAA -= turn/(pad*pad)
  return AAA

def feature2(state, action):
  charger = np.array([Height-3,Width-3])
  life = state[3]
  pos = action()
  turn = isinstance(pos, float)
  if turn:
    pos=possible_actions[0](pos)
  else:
    life -= - 1/500
  distManhattan = charger-pos
  distManhattan = np.sum(np.abs(distManhattan))
  #print(min((1-life)/distManhattan,1))
  return min((1-life)/distManhattan,1)

def feature3(state, action):
  dirtMap = state[4:].reshape(20,20)
  pos = action()
  turn = isinstance(pos, float)
  if turn:
    pos=possible_actions[0](pos)
  BBB = closestDirt(int(pos[0]),int(pos[1]), dirtMap)/(Height+Width)
  BBB -= turn/(Height+Width)
  return BBB

def feature4(state, action):
  return 1

In [48]:
#TODO: try to add more lookahead for next_Step

# Hyperparameters
alpha = 0.1
gamma = 0.9999
epsilon = 0.3

weights = [0, 0, 0, 0]
F = [feature1, feature2, feature3, feature4]

def getQValue(state, action):
  qv = 0
  for i, feature in enumerate(F):
      qv = qv + weights[i] * feature(state, action)
  return qv

for e in range(5):
  done = False
  state = env.reset()
  while not done:
    if random.uniform(0, 1) < epsilon:
        action = env.action_space.sample()
    else:
        q = np.array([getQValue(state, a) for a in possible_actions])
        action = np.argmax(q)

    next_state, reward, done, info = env.step(action)

    action = possible_actions[action]

    next_values = [getQValue(next_state, a) for a in possible_actions]
    next_max = max(next_values)

    alphadiff = alpha * ((reward + gamma * next_max) - getQValue(state, action))
    
    for i, feature in enumerate(F):
        weights[i] = weights[i] + alphadiff * feature(state, action)

    state = next_state

state = env.reset()
while not done:
  env.render()
  q = np.array([getQValue(state, a) for a in possible_actions])
  action = np.argmax(q)

  next_state, reward, done, info = env.step(action)

  action = possible_actions[action]

  next_values = [getQValue(next_state, a) for a in possible_actions]
  next_max = max(next_values)

  alphadiff = alpha * ((reward + gamma * next_max) - getQValue(state, action))
  
  for i, feature in enumerate(F):
      weights[i] = weights[i] + alphadiff * feature(state, action)

  state = next_state

env.close()
show_video()

In [49]:
weights

[18275511.093110994,
 11159062.888008112,
 246576.62346853435,
 14100846.127032835]

In [56]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

while True:
  
    env.render()
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action)
    
    if done:
      break;
            
env.close()
show_video()

If the goal is to train on an unchanging room, then no need for giving walls/charging station as input? He will learn automatically?


In [38]:

import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, GreedyQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory



env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse

nb_actions = env.action_space.n


# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=60000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10000,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

# After training is done, we save the final weights.
dqn.save_weights('dqn_roomba_weights.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=5, visualize=False)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 404)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                25920     
_________________________________________________________________
activation_17 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                2080      
_________________________________________________________________
activation_18 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_19 (Dense)             (None, 16)                528       
_________________________________________________________________
activation_19 (Activation)   (None, 16)                0         
__________

In [39]:
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 10s 1ms/step - reward: -1.4229
8 episodes - episode_reward: -1721.125 [-3134.000, -1193.000]

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -1.5815
10 episodes - episode_reward: -1599.300 [-2046.000, -1121.000] - loss: 89.378 - mean_absolute_error: 43.874 - mean_q: -54.465

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 50s 5ms/step - reward: -1.4804
9 episodes - episode_reward: -1629.000 [-2784.000, -1086.000] - loss: 68.105 - mean_absolute_error: 53.546 - mean_q: -64.663

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 49s 5ms/step - reward: -1.4637
9 episodes - episode_reward: -1668.222 [-2843.000, -1182.000] - loss: 51.038 - mean_absolute_error: 57.728 - mean_q: -71.874

Interval 5 (40000 steps performed)
10000/10000 [==============================] -

In [40]:
! rm -rf video/

env = wrap_env(RoombaEnv(20, 20, 500, 2))

dqn.test(env, nb_episodes=1, visualize=True)

            
env.close()
show_video()

Testing for 1 episodes ...
Episode 1: reward: -1611.000, steps: 884


In [41]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory



# Get the environment and extract the number of actions.
env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse


nb_actions = env.action_space.n
obs_dim = env.observation_space.shape[0]

# Option 1 : Simple model
#model = Sequential()
#model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
#model.add(Dense(nb_actions))
#model.add(Activation('softmax'))

# Option 2: deep network
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('softmax'))


print(model.summary())


# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = EpisodeParameterMemory(limit=100000, window_length=1)

cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=5000, nb_steps_warmup=10000, train_interval=50, elite_frac=0.05)
cem.compile()

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
cem.fit(env, nb_steps=200000, visualize=False, verbose=1)

# After training is done, we save the best weights.
cem.save_weights('cem_roomba_params.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
cem.test(env, nb_episodes=5, visualize=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 404)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 16)                6480      
_________________________________________________________________
activation_21 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_22 (Activation)   (None, 16)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 16)                272       
_________________________________________________________________
activation_23 (Activation)   (None, 16)                0         
__________

/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 10s 952us/step - reward: -1.4393
8 episodes - episode_reward: -1754.750 [-2498.000, -1233.000] - mean_best_reward: -1162.000

Interval 7 (60000 steps performed)
10000/10000 [==============================] - 10s 968us/step - reward: -1.6061
10 episodes - episode_reward: -1647.100 [-3379.000, -1134.000]

Interval 8 (70000 steps performed)
10000/10000 [==============================] - 10s 966us/step - reward: -1.5724
10 episodes - episode_reward: -1481.400 [-2550.000, -1186.000]

Interval 9 (80000 steps performed)
10000/10000 [==============================] - 9s 949us/step - reward: -1.6984
11 episodes - episode_reward: -1591.727 [-2774.000, -1203.000]

Interval 10 (90000 steps performed)
10000/10000 [==============================] - 9s 933us/step - reward: -1.4478
8 episodes - episode_reward: -1812.875 [-3624.000, -1188.000]

Interval 11 (100000 steps performed)
10000/10000 [==============================] - 10s 955us/step - reward: -1.8

In [42]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

cem.test(env, nb_episodes=1, visualize=True)

            
env.close()
show_video()

Testing for 1 episodes ...
Episode 1: reward: -1250.000, steps: 502


In [0]:
random.random()

In [43]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents import SARSAAgent
from rl.policy import BoltzmannQPolicy



# Get the environment and extract the number of actions.
env = RoombaEnv(20, 20, 500, 2) #wrap env seems to grow exponentially with H,W?? Maybe scalse
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

# SARSA does not require a memory.
policy = BoltzmannQPolicy()
sarsa = SARSAAgent(model=model, nb_actions=nb_actions, nb_steps_warmup=10000, policy=policy)
sarsa.compile(Adam(lr=1e-3), metrics=['mae'])

# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
sarsa.fit(env, nb_steps=100000, visualize=False, verbose=2)

# After training is done, we save the final weights.
sarsa.save_weights('sarsa_roomba_weights.h5f', overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
sarsa.test(env, nb_episodes=5, visualize=False)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 404)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 64)                25920     
_________________________________________________________________
activation_25 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 32)                2080      
_________________________________________________________________
activation_26 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 16)                528       
_________________________________________________________________
activation_27 (Activation)   (None, 16)                0         
__________

In [44]:
#env = wrap_env(RoombaEnv(10, 10, 2))
env = wrap_env(RoombaEnv(20, 20, 500, 2)) #wrap env seems to grow exponentially with H,W?? Maybe scalse
#env = wrap_env(gym.make("CartPole-v0"))
observation = env.reset()

sarsa.test(env, nb_episodes=1, visualize=True)

            
env.close()
show_video()

Testing for 1 episodes ...
Episode 1: reward: -10000.000, steps: 10000


In [0]:
np.random.choice(400, p=dirt_probabilities)

In [0]:
dirt_probabilities.shape